In [31]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_mistralai import ChatMistralAI
from langchain_google_genai import ChatGoogleGenerativeAI

/Users/pramodmodi/Library/Caches/pypoetry/virtualenvs/reflection-agent-J-iJ8P-G-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
mistral_api_key = os.environ["MISTRAL_API_KEY"]
google_api_key = os.environ["GOOGLE_API_KEY"]

In [3]:
# llm = ChatMistralAI(
#     mistral_api_key=mistral_api_key,
#     model="mistral-large-latest",
#     temperature=0,
#     max_retries=2,
#     # other params...


In [89]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

In [91]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a viral twitter influencer grading a tweet. Generate critique and recommendations for the user's tweet."
            "Always provide detailed recommendations, including requests for length, virality, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
            " Generate the best twitter post possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [93]:
generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

In [95]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import END, MessageGraph
from typing import List, Sequence

## Two nodes

In [98]:

REFLECT = "reflect"
GENERATE = "generate"

In [129]:
def generation_node(state: Sequence[BaseMessage]):
    #res = generate_chain.invoke({"messages": state})
    #return [res.content]
   # return [AIMessage(content=generate_chain.invoke({"messages": state}))]
    return generate_chain.invoke({"messages": state})


def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflect_chain.invoke({"messages": messages})
    return [HumanMessage(content=res.content)]

## Build Graph

In [132]:

builder = MessageGraph()
builder.add_node(GENERATE, generation_node)
builder.add_node(REFLECT, reflection_node)
builder.set_entry_point(GENERATE)


In [134]:
# Add edge betwee reflect and generate.
builder.add_edge(REFLECT, GENERATE)

In [136]:

## This is conditional edge. Should continue method give feedback 6 times, then end it.

def should_continue(state: List[BaseMessage]):
    if len(state) > 3:
        return END
    return REFLECT

In [138]:

builder.add_conditional_edges(GENERATE, should_continue)


In [140]:
graph = builder.compile()

In [142]:
inputs = HumanMessage(content="""Make this tweet better:"
                                    @LangChainAI
            — newly Tool Calling feature is seriously underrated.

            After a long wait, it's  here- making the implementation of agents across different models with function calling - super easy.

            Made a video covering their newest blog post

                                  """)
response = graph.invoke(inputs)

In [143]:
response

[HumanMessage(content='Make this tweet better:"\n                                    @LangChainAI\n            — newly Tool Calling feature is seriously underrated.\n\n            After a long wait, it\'s  here- making the implementation of agents across different models with function calling - super easy.\n\n            Made a video covering their newest blog post\n\n                                  ', additional_kwargs={}, response_metadata={}, id='611a1c5e-08e8-4664-a1ba-3ec3fea53a93'),
 AIMessage(content="🚀 LangChain's new Tool Calling feature is a game-changer! 🤩  Implementing agents across different models with function calling just got WAY easier.  Check out my video breakdown of their latest blog post! #LangChain #AI #ToolCalling #Agents  [Link to Video]\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-e61ca62c-6610-4122-89de-4d636a37c075-0', usage_metadata={'input

In [146]:
inputs = HumanMessage(content="""Make this tweet better:"
                                   LLM came up with beam of data and solution. Langchaion is carrier to the LLM solution.

                                  """)
response = graph.invoke(inputs)

In [147]:
response

[HumanMessage(content='Make this tweet better:"\n                                   LLM came up with beam of data and solution. Langchaion is carrier to the LLM solution.\n\n                                  ', additional_kwargs={}, response_metadata={}, id='50ecb742-887e-474e-b0f9-869207dfba4a'),
 AIMessage(content='LangChain 🤝 LLMs:  Think of LLMs as the powerhouse generating brilliant insights, and LangChain as the delivery system, bringing those solutions straight to your applications. #LangChain #LLMs #AI #Data\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-af385083-b5be-4799-92f1-842c44f51950-0', usage_metadata={'input_tokens': 78, 'output_tokens': 46, 'total_tokens': 124, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='**Critique:**\n\n* **Jargon:** The original tweet uses terms like "beam of data" which, while evocative, might not be universally